In [1]:
import pandas as pd
import numpy as np

In [2]:
# Reading the sales data CSV file and converting specific date columns to datetime format for easier manipulation
df = pd.read_csv('quarterly_df.csv', low_memory=False)

In [3]:
# Function to extract quarter and year from a date
def get_quarter_year_from_date(date):
    # Subtract one day from the date to get the correct quarter
    date = pd.to_datetime(date) - pd.Timedelta(days=1)
    return str(date.year) + 'Q' + str(date.quarter)

df['pred_close_q'] = df['Opportunity_Close_Date'].apply(get_quarter_year_from_date)

In [9]:
# Initializing a new column for the true closing quarter of opportunities
df['close_won_q'] = None

# Iterating over each row in DataFrame to determine the actual closing quarter based on certain conditions
for index, curr_row in df.iterrows():
    curr_id = curr_row['Opportunity_ID']
    curr_opp_df = df[df['Opportunity_ID'] == curr_id].copy()

    closed_row = curr_opp_df[(curr_opp_df['Opportunity_Stage'] == 'Closed Won')]
    
    if len(closed_row) > 0:
        true_close_q = closed_row['Year_Quarter'].values[0]
        df.loc[index, 'close_won_q'] = true_close_q

# Function to calculate the next quarter from a given quarter-year string
def add_one_quarter(date_str):
    if date_str == None:
        return None

    quarter = int(date_str[-1])
    year = int(date_str[0:4])
    if quarter == 4:
        quarter = 1
        year += 1
    else:
        quarter += 1
    return str(year) + 'Q' + str(quarter)

# Applying the function to create a new column for the next quarter
df['next_quarter'] = df['Year_Quarter'].apply(add_one_quarter)

In [10]:
df['q_1'] = df['next_quarter'] == df['close_won_q']
df['q_1'] = df['q_1'].astype(int)

df['q_1'].mean()

0.04078212290502793

In [11]:
df['q2_a'] = df['pred_close_q'] == df['close_won_q']
df['q2_a'] = df['q2_a'].astype(int)

df['q2_b'] = (df['pred_close_q'] == df['close_won_q']) | \
                (df['pred_close_q'] == df['close_won_q'].apply(add_one_quarter))
df['q2_b'] = df['q2_b'].astype(int)